In [1]:
import osmnx as ox
import matplotlib.pyplot as plt
import networkx as nx
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import math

from shapely.geometry import box, Point, LineString
from heapq import heappush, heappop
from itertools import count

In [2]:
place_name = "Singapore, Central, Singapore"
# G = ox.graph_from_place(place_name, network_type='drive')
# ox.save_load.save_graphml(G, filename='Singapore_drive.graphml', folder=None, gephi=False)

graph = ox.save_load.load_graphml(filename="Singapore_drive.graphml", folder=None)

# Stores nodes and edges along with other attributes
type(graph)

networkx.classes.multidigraph.MultiDiGraph

In [3]:
nodes, edges = ox.graph_to_gdfs(graph, nodes=True, edges=True)
#to save the dataframe
nodes.to_pickle('data/nodes.pkl')   

In [4]:
def process_edges(edges):
    # Remove null, list, and convert to int ---> maxspeed (km/hr)
    edges['maxspeed'].fillna('40', inplace=True)

    i = 0
    for x in edges['maxspeed']:
        if type(x) == list:
            edges['maxspeed'][i] = max(x)
        i += 1    

    edges['maxspeed'] = edges['maxspeed'].astype(float)
    

    # Define free-flow time t(u,v) in sec
    travel_time = []
    for l,s in zip(edges['length'], edges['maxspeed']):
        t = (3600*l)/(1000*s)
        travel_time.append(t)

    edges['travel_time'] = travel_time    
    
    # Drop unnecessary columns
    edges.drop(['lanes','bridge','tunnel','ref','junction','access','service','width'], axis=1, inplace=True)

In [5]:
process_edges(edges)
edges.head()

/home/orionstar25/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,u,v,key,osmid,oneway,name,highway,maxspeed,length,travel_time,Location
0,1820000257,1865254946,0,174765824,True,Tampines Avenue 8,tertiary,60.0,14.248,0.85488,"(1.3565805, 103.9326163, 1.3566943, 103.9326752)"
1,1820000257,4662056865,0,587044201,True,Tampines Avenue 5,primary,60.0,27.282,1.63692,"(1.3565805, 103.9323996, 1.3566955, 103.9326163)"
2,3874553858,6995781882,0,384254693,False,Fidelio Street,tertiary,40.0,20.740,1.86660,"(1.3191823, 103.9212368, 1.3192215, 103.9214192)"
3,3874553858,3874553861,0,384254631,True,Fidelio Street,tertiary,40.0,15.191,1.36719,"(1.3191111, 103.9211207, 1.3191823, 103.9212368)"
4,1820000259,245190289,0,170861145,True,Tampines Avenue 8,tertiary,60.0,13.573,0.81438,"(1.3565404, 103.9326858, 1.3566471, 103.9327451)"


In [6]:
G = ox.gdfs_to_graph(nodes, edges)
ox.save_load.save_graphml(G, filename='Singapore_processed.graphml', folder=None, gephi=False)

In [1]:
def euclidean_dist(nodes):
    #key: (u,v) value: dist 
    # euc_dist = dict()
    lat = nodes['y']
    long = nodes['x']
    ids = nodes['osmid']

    with open('euclidean_distances.json', 'a') as f:
        f.write('{\n')

        for lat_1, lon_1, u in zip(lat,long,ids):
            for lat_2, lon_2, v in zip(lat,long,ids):
                # distance in metres
                d = 1000 * ((lat_1-lat_2)**2 + (lon_1-lon_2)**2)**0.5
                content = "({},{}) : {}\n"
                print(content.format(u,v,d))
                f.write(content.format(u,v,d))
                f.write(content.format(v,u,d))
        f.write('}\n')

In [ ]:
euclidean_dist(nodes)

In [2]:
nodes = pd.read_pickle('data/nodes.pkl')

In [13]:
nodes['highway'].unique()

array(['traffic_signals', nan, 'turning_circle', 'crossing',
       'motorway_junction', 'turning_loop', 'speed_camera',
       'mini_roundabout', 'stop'], dtype=object)